In [1]:
import os
# Display current working directory
print(os.getcwd())
# To make sure opencv imports .exr files
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
# If the current directory is not WDSS, then set it to one level up
if os.getcwd()[-4:] != 'WDSS':
    os.chdir('..')
print(os.getcwd())

c:\Dev\MinorProject\WDSS\jupyter_notebooks
c:\Dev\MinorProject\WDSS


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

from typing import List, Tuple, Dict

from utils.image_utils import ImageUtils
from config import device, Settings
from commons import initialize

In [8]:
from network.dataset import *
import tqdm

In [27]:
def pack_metallic_roughness_specular(path: str, total_frames: int, save_file_name: str = 'MetallicRoughnessSpecular'):
    for i in tqdm.tqdm(range(total_frames), desc='Packing Metallic, Roughness and Specular'):
        metallic_path = f'{path}\\Metallic'
        roughness_path = f'{path}\\Roughness'
        normal_path = f'{path}\\Specular'
        idx = "." + str(i).zfill(4) + ".exr"

        metallic = ImageUtils.load_exr_image_opencv(metallic_path + idx)[:, :, 0:1]
        roughness = ImageUtils.load_exr_image_opencv(roughness_path + idx)[:, :, 0:1]
        normal = ImageUtils.load_exr_image_opencv(normal_path + idx)[:, :, 0:1]

        packed = np.concatenate((metallic, roughness, normal), axis=2)

        ImageUtils.save_exr_image_opencv(packed, f'{path}\\{save_file_name}{idx}')

def pack_nov_depth(path: str, total_frames: int, save_file_name: str = 'NoVDepth'):
    for i in tqdm.tqdm(range(total_frames), desc='Packing NoV and Depth'):
        nov_path = f'{path}\\Nov'
        depth_path = f'{path}\\SceneDepth'
        idx = "." + str(i).zfill(4) + ".exr"

        nov = ImageUtils.load_exr_image_opencv(nov_path + idx)[:, :, 0:1]
        depth = ImageUtils.load_exr_image_opencv(depth_path + idx)[:, :, 0:1]

        packed = np.concatenate((nov, depth, depth), axis=2)

        ImageUtils.save_exr_image_opencv(packed, f'{path}\\{save_file_name}{idx}')

def delete_files(path: str, total_frames: int, file_name: str):
    for i in tqdm.tqdm(range(total_frames), desc=f'Deleting {file_name}'):
        idx = "." + str(i).zfill(4) + ".exr"
        os.remove(f'{path}\\{file_name}{idx}')

In [28]:
def pack_g_buffer(path: str, total_frames: int):
    pack_metallic_roughness_specular(path, total_frames)
    pack_nov_depth(path, total_frames)
    delete_files(path, total_frames, 'Metallic')
    delete_files(path, total_frames, 'Roughness')
    delete_files(path, total_frames, 'Specular')
    delete_files(path, total_frames, 'Nov')
    delete_files(path, total_frames, 'SceneDepth')

In [29]:
def pack_dataset(path: str):
    pack_g_buffer(f'{path}\\LowResGbuffer', 127)
    pack_g_buffer(f'{path}\\HighResGbuffer', 127)

In [30]:
path = 'D:\\Dev\\MinorProjDataset\\V4\\Data\\train\\Asain_Village_1'

In [31]:
pack_dataset(path)

Deleting SceneDepth: 100%|██████████| 127/127 [00:00<00:00, 611.06it/s]
